## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 4
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_4


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 20.5 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Detroit Lions,4,2022,35.0,66.7,7.4,272.8,100.0,12.2,35.3,...,12.2,165.5,10.75,13.25,13.25,0.076190,2.75,2.5,0.70,3.666667
1,Philadelphia Eagles,4,2022,28.8,63.6,9.1,270.3,99.6,13.7,17.8,...,9.8,100.3,28.25,15.00,12.50,0.105263,1.00,1.0,1.25,2.000000


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[4]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
4,Kansas City Chiefs,2022,week_4,0.695354,0.189624,0.173026,0.286193,0.416284,0.352096,0.451168
2,Buffalo Bills,2022,week_4,0.626645,0.304943,0.121424,0.270087,0.376066,0.339833,0.407388
1,Philadelphia Eagles,2022,week_4,0.561078,0.239004,0.147493,0.255041,0.365472,0.313617,0.389922


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [10]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
6,Kansas City Chiefs,2022,week_3,0.951682,0.299390,0.244908,0.407212,0.435774,0.467793,0.500262
0,Philadelphia Eagles,2022,week_3,0.907062,0.380422,0.187154,0.348187,0.390039,0.442573,0.454667
1,Buffalo Bills,2022,week_3,0.800414,0.299441,0.130100,0.255513,0.378469,0.372787,0.431212
12,Miami Dolphins,2022,week_3,0.094912,0.277060,0.189605,0.348634,0.395857,0.261214,0.358238
5,Jacksonville Jaguars,2022,week_3,0.156894,0.215884,0.134501,0.318645,0.370865,0.239358,0.344119
26,Tampa Bay Buccaneers,2022,week_3,0.178198,0.195837,0.100057,0.215016,0.365509,0.210923,0.342095
3,Baltimore Ravens,2022,week_3,0.072906,0.216870,0.122034,0.261517,0.374225,0.209511,0.336560
11,Green Bay Packers,2022,week_3,0.012405,0.179944,0.126746,0.289279,0.367143,0.195103,0.322801
10,Los Angeles Chargers,2022,week_3,0.001206,0.128269,0.087304,0.263125,0.366910,0.169362,0.321197
29,Carolina Panthers,2022,week_3,0.117355,0.107779,0.074643,0.189097,0.345883,0.166951,0.317317
